Here is a sample code for gru4rec using recbole. 
I made it based on the following code from a past competition
https://www.kaggle.com/code/astrung/recbole-lstm-sequential-for-recomendation-tutorial

I think you can get a better score if you change the training data or epoch size, etc. Enjoy!

In [1]:
!pip install polars

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install recbole

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 24.5 MB/s eta 0:00:00


In [3]:
!pip install ray

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 98.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.5/468.5 kB 25.5 MB/s eta 0:00:00
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.54.0
    Uninstalling grpcio-1.54.0:
      Successfully uninstalled grpcio-1.54.0


In [11]:
!unzip sessions.zip

Archive:  sessions.zip
  inflating: sessions.jsonl          


In [12]:
!pip install hyperopt==0.2.5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
import tqdm
import polars as pl
import numpy as np
import pandas as pd
import seaborn as sns
import random
import os 
import h5py
import sys
import gc

from matplotlib import pyplot as plt
import pyarrow.parquet as pq

# 1. Przygotowanie danych

In [14]:
sessions_df = pd.read_json('sessions.jsonl')
sessions_df

,session_id,timestamp,user_id,track_id,event_type
0,124,2022-07-19 17:17:13.000,101,7b7AziuX7EeIzvatDcC0RS,play
1,124,2022-07-19 17:20:33.880,101,5RubKOuDoPn5Kj5TLVxSxY,play
2,124,2022-07-19 17:22:48.894,101,23zzKOubAzAu8xlQdkVvZ2,play
3,124,2022-07-19 17:28:23.521,101,2DVkYfl2YuOMlRfC98srEG,play
4,124,2022-07-19 17:32:25.619,101,5wTuWXWREngkFGtBXhOaI9,play
...,...,...,...,...,...
1279798,60198,2022-10-19 02:22:52.276,4100,3tSuRiNgJmMqmYR23dCqnR,play
1279799,60198,2022-10-19 02:23:40.099,4100,3tSuRiNgJmMqmYR23dCqnR,like
1279800,60198,2022-10-19 02:26:35.676,4100,None,advertisment
1279801,60198,2022-10-19 02:26:35.676,4100,2RRFTLIglxHgAYQzsSLI4Y,play


In [15]:
sessions_df.dropna(inplace=True)
sessions_df.drop('session_id', axis=1, inplace=True)
sessions_df.drop('event_type', axis=1, inplace=True)
sessions_df

,timestamp,user_id,track_id
0,2022-07-19 17:17:13.000,101,7b7AziuX7EeIzvatDcC0RS
1,2022-07-19 17:20:33.880,101,5RubKOuDoPn5Kj5TLVxSxY
2,2022-07-19 17:22:48.894,101,23zzKOubAzAu8xlQdkVvZ2
3,2022-07-19 17:28:23.521,101,2DVkYfl2YuOMlRfC98srEG
4,2022-07-19 17:32:25.619,101,5wTuWXWREngkFGtBXhOaI9
...,...,...,...
1279797,2022-10-19 02:18:24.943,4100,7dWSrvzawBZPa7VVeHZBn7
1279798,2022-10-19 02:22:52.276,4100,3tSuRiNgJmMqmYR23dCqnR
1279799,2022-10-19 02:23:40.099,4100,3tSuRiNgJmMqmYR23dCqnR
1279801,2022-10-19 02:26:35.676,4100,2RRFTLIglxHgAYQzsSLI4Y


In [16]:
import datetime as dt
sessions_df['timestamp'] = pd.to_datetime(sessions_df['timestamp'])
sessions_df['timestamp'] = (sessions_df['timestamp'] - dt.datetime(1970,1,1)).dt.total_seconds().astype('int64')
sessions_df

,timestamp,user_id,track_id
0,1658251033,101,7b7AziuX7EeIzvatDcC0RS
1,1658251233,101,5RubKOuDoPn5Kj5TLVxSxY
2,1658251368,101,23zzKOubAzAu8xlQdkVvZ2
3,1658251703,101,2DVkYfl2YuOMlRfC98srEG
4,1658251945,101,5wTuWXWREngkFGtBXhOaI9
...,...,...,...
1279797,1666145904,4100,7dWSrvzawBZPa7VVeHZBn7
1279798,1666146172,4100,3tSuRiNgJmMqmYR23dCqnR
1279799,1666146220,4100,3tSuRiNgJmMqmYR23dCqnR
1279801,1666146395,4100,2RRFTLIglxHgAYQzsSLI4Y


In [17]:
sessions_df.rename(columns={"timestamp" : "timestamp:float",
                    "user_id":"user_id:token", 
                    "track_id": "item_id:token"}, inplace=True)
sessions_df

,timestamp:float,user_id:token,item_id:token
0,1658251033,101,7b7AziuX7EeIzvatDcC0RS
1,1658251233,101,5RubKOuDoPn5Kj5TLVxSxY
2,1658251368,101,23zzKOubAzAu8xlQdkVvZ2
3,1658251703,101,2DVkYfl2YuOMlRfC98srEG
4,1658251945,101,5wTuWXWREngkFGtBXhOaI9
...,...,...,...
1279797,1666145904,4100,7dWSrvzawBZPa7VVeHZBn7
1279798,1666146172,4100,3tSuRiNgJmMqmYR23dCqnR
1279799,1666146220,4100,3tSuRiNgJmMqmYR23dCqnR
1279801,1666146395,4100,2RRFTLIglxHgAYQzsSLI4Y


In [18]:
sessions_df = sessions_df[
    [
        "user_id:token",
        "item_id:token",
        "timestamp:float"
    ]
]
sessions_df

,user_id:token,item_id:token,timestamp:float
0,101,7b7AziuX7EeIzvatDcC0RS,1658251033
1,101,5RubKOuDoPn5Kj5TLVxSxY,1658251233
2,101,23zzKOubAzAu8xlQdkVvZ2,1658251368
3,101,2DVkYfl2YuOMlRfC98srEG,1658251703
4,101,5wTuWXWREngkFGtBXhOaI9,1658251945
...,...,...,...
1279797,4100,7dWSrvzawBZPa7VVeHZBn7,1666145904
1279798,4100,3tSuRiNgJmMqmYR23dCqnR,1666146172
1279799,4100,3tSuRiNgJmMqmYR23dCqnR,1666146220
1279801,4100,2RRFTLIglxHgAYQzsSLI4Y,1666146395


In [19]:
!mkdir ./sessions
sessions_df[['user_id:token', 'item_id:token', 'timestamp:float']].to_csv("./sessions/sessions.inter", index=False, sep='\t')

del sessions_df
gc.collect()

0

# 2. Trening i przeszukiwanie przestrzeni hiperparametrów

In [20]:
import logging
from logging import getLogger
from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.model.sequential_recommender import GRU4Rec
from recbole.trainer import Trainer
from recbole.utils import init_seed, init_logger

from recbole.utils.case_study import full_sort_topk
from recbole.quick_start import objective_function

In [21]:
from ray import tune
from recbole.config import Config
from recbole.data import (
    create_dataset,
    data_preparation,
    save_split_dataloaders,
    load_split_dataloaders,
)
from recbole.data.transform import construct_transform
from recbole.utils import (
    init_logger,
    get_model,
    get_trainer,
    init_seed,
)

In [22]:
init_seed(0,True)

In [23]:
from recbole.trainer.hyper_tuning import HyperTuning
hp = HyperTuning(objective_function=objective_function, algo='exhaustive',
                params_file='hyper.test', fixed_config_file_list=['example.yaml'])
hp.run()

running parameters:
{'hidden_size': 128, 'learning_rate': 0.005, 'num_layers': 1}
  0%|          | 0/45 [00:00<?, ?trial/s, best loss=?]

19 May 19:05    INFO  [Training]: train_batch_size = [1024] train_neg_sample_args: [{'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}]

19 May 19:05    INFO  [Evaluation]: eval_batch_size = [1024] eval_args: [{'split': {'RS': [8, 1, 1]}, 'group_by': 'user', 'order': 'TO', 'mode': 'full'}]

19 May 19:08    INFO  Loading model structure and parameters from saved/GRU4Rec-May-19-2023_19-05-25.pth

19 May 19:08    WARNING  Session not detected. You should not be calling `report` outside `tuner.fit()` or while using the class API. 

19 May 19:08    WARNING    File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.10/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.10/dist-packages/traitlets/c

current best valid score: 0.3874
current best valid result:
OrderedDict([('recall@20', 0.3874), ('mrr@20', 0.3798), ('ndcg@20', 0.3814), ('hit@20', 0.3874), ('precision@20', 0.0194)])
current test result:
OrderedDict([('recall@20', 0.3881), ('mrr@20', 0.381), ('ndcg@20', 0.3825), ('hit@20', 0.3881), ('precision@20', 0.0194)])
running parameters:
{'hidden_size': 64, 'learning_rate': 0.005, 'num_layers': 1}
  2%|▏         | 1/45 [03:49<2:48:39, 229.98s/trial, best loss: -0.3874]

19 May 19:09    INFO  [Training]: train_batch_size = [1024] train_neg_sample_args: [{'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}]

19 May 19:09    INFO  [Evaluation]: eval_batch_size = [1024] eval_args: [{'split': {'RS': [8, 1, 1]}, 'group_by': 'user', 'order': 'TO', 'mode': 'full'}]

19 May 19:11    INFO  Loading model structure and parameters from saved/GRU4Rec-May-19-2023_19-09-05.pth



running parameters:
{'hidden_size': 128, 'learning_rate': 0.001, 'num_layers': 1}
  4%|▍         | 2/45 [06:48<2:22:58, 199.50s/trial, best loss: -0.3874]

...

running parameters:
{'hidden_size': 128, 'learning_rate': 0.0005, 'num_layers': 3}
 18%|█▊        | 8/45 [31:19<2:34:51, 251.13s/trial, best loss: -0.3874]

19 May 19:36    INFO  [Training]: train_batch_size = [1024] train_neg_sample_args: [{'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}]

19 May 19:36    INFO  [Evaluation]: eval_batch_size = [1024] eval_args: [{'split': {'RS': [8, 1, 1]}, 'group_by': 'user', 'order': 'TO', 'mode': 'full'}]

19 May 19:40    INFO  Loading model structure and parameters from saved/GRU4Rec-May-19-2023_19-36-35.pth



running parameters:
{'hidden_size': 128, 'learning_rate': 0.0005, 'num_layers': 1}
 20%|██        | 9/45 [35:52<2:34:49, 258.03s/trial, best loss: -0.3874]

19 May 19:41    INFO  [Training]: train_batch_size = [1024] train_neg_sample_args: [{'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}]

19 May 19:41    INFO  [Evaluation]: eval_batch_size = [1024] eval_args: [{'split': {'RS': [8, 1, 1]}, 'group_by': 'user', 'order': 'TO', 'mode': 'full'}]

19 May 19:44    INFO  Loading model structure and parameters from saved/GRU4Rec-May-19-2023_19-41-08.pth

19 May 19:44    INFO  Early stop triggered. Stopping iterations as condition is reach.



 22%|██▏       | 10/45 [39:30<2:18:18, 237.09s/trial, best loss: -0.3874]


In [24]:
hp.display_file = 'display'
hp.plot_hyper()

/usr/local/lib/python3.10/dist-packages/plotly/offline/offline.py:562: UserWarning:

Your filename `display` didn't end with .html. Adding .html to the end of your file.

